In this lab we will go over over basic stochastic optimization and uses in pytorch



(1) Setup the MNIST dataloaders for both the training (and  now as well test) set as in Lab 1 part 2. You do not need to iterate through the dataloaders, these will be used in the rest of the lab. 


In [4]:
import numpy as np
import math
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import timeit

In [5]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [6]:
x= torch.randn(5 , requires_grad=True)
y=x**2 
y.backward()
y=x**3 
y.backward()
print(x.grad)

RuntimeError: grad can be implicitly created only for scalar outputs

In [7]:
batch_size_train = 256
batch_size_test = 256

In [8]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)
print(example_data.shape)

torch.Size([256, 1, 28, 28])


(2) Consider your work from [Lab 1 part 2 (2)]. Modify the neural network as follows. The hidden outputs should be size 100 in both hidden layers and the activations should be relu. Modify the final layer to output 10 values instead of 1. 

Your code should implement $f(x) = W_2\rho(W_1\rho(W_0x+b_0)+b_1)+b_2$ with $f: R^{786}-> R^{10}$ and $\rho = relu$

Initialize the weights  using a variant of xavier intialization $w_{ij} \sim N(0,\frac{1}{\sqrt{n_i}})$ where $n_i$ is the size of the layer input. Initialize the biases as 0. Write a helper function to perform this initialization for subsequent parts of this lab

In [9]:
def xavier(m,h):
    return np.random.randn(m,h)*np.math.sqrt(1/m)
xavier(100,100)

array([[-0.14483055,  0.20985911,  0.15247459, ..., -0.0756262 ,
         0.11198171,  0.02941895],
       [-0.12859168,  0.2728823 ,  0.04559954, ..., -0.02268817,
         0.0892945 ,  0.04191412],
       [ 0.19581739, -0.05347473, -0.11650499, ...,  0.13047089,
        -0.04795322,  0.16375934],
       ...,
       [ 0.06831523,  0.06099567, -0.05386164, ...,  0.11614697,
         0.06132986,  0.06845243],
       [ 0.12989187,  0.06402156,  0.10899956, ...,  0.18078908,
        -0.12812119,  0.20618821],
       [ 0.0550139 , -0.02729148, -0.20291188, ...,  0.22803449,
         0.08416595, -0.03106789]])

In [10]:
param_dict={
    # print(W1)
    "W1": torch.randn((20, 10), requires_grad=True)
    "W2": torch.randn((20, 1), requires_grad=True)
    # define the bias terms
    "B1":torch.randn((20), requires_grad=True)
    "B2":torch.randn((20), requires_grad=True)
    }

In [11]:
## Define the network
def my_nn(input,param_dict): 
    W1=param_dict["W1"]
    W2=param_dict["W2"]
    B1=param_dict["B1"]
    features = torch.flatten(input).reshape(-1)
        # calculate hidden and output layers
    h1 = torch.tanh_((features @ W0) + B0)
    h2 = torch.relu_((h1 @ W1) + B1)
    output = h2 @ W2 + B2
#     output=(output - 2)/0.5
#     print(output)
#     print(output.reshape(-1,1))
    return output
#      return F.log_softmax(output.reshape(-1,1), dim=1)

start = timeit.default_timer()   
for idx in range(len(example_data)):
     my_nn(example_data[idx][0],param_dict)
stop = timeit.default_timer()
print('FF 256 images Time= :',stop - start) 
            


FF 256 images Time= : 0.7837040959999939


We will evaluate the cross entropy loss and average accuracy on this randomly initialized dataset. Use the torch.nn.functional.cross_entropy() function to compute the loss  $\frac{1}{N}\sum_i^N CrossEntropy(f(x_i),y_i)$ and accuracy. Your accuracy should be close to $10\%$ as the network has random weights. Note the pytorch cross_entropy function already applies the softmax operator so you do not need to apply this just feed the model output directly  



In [178]:
from math import log2
def cross_entropy(p, q):
    return -sum([p[i]*log2(q[i]) for i in range(len(p))])

In [228]:
from itertools import repeat 
cuda = torch.cuda.is_available()
acc=[]
loss=[]
batch_idx=0
for data, targets in test_loader:
  #move to GPU if available
#   if cuda:
#     data, target = data.to('cuda'), target.to('cuda')
  #compute model output
#     print(len(data))
#     for i in range(len(data)):
    Q=torch.zeros(len(data), 10)
    for i in range(len(data)):
        out=my_nn(data[i],param_dict)
#         print(out.numpy())
        Q[i,:]=out
#         print(len(targets))
  #comptue accuracy for minibatch
    pred = torch.nn.functional.softmax(Q, dim=1)
    total=0
    for i, p in enumerate(pred):
        if targets[i] == torch.max(p.data, 0)[1]:
            total = total + 1
    acc.append(total/len(data))
  #compute loss for minibatch
    loss.append(torch.nn.functional.cross_entropy(Q,targets))
    print('batch: ',batch_idx,' loss=',loss[batch_idx],' acc=',acc[batch_idx])
    batch_idx=batch_idx+1
#aggregate loss and accuracy for all test data
print('avg acc =',sum(acc)/len(acc))
print('avg loss =',sum(loss)/len(loss))  

batch:  0  loss= tensor(2.3805)  acc= 0.07421875
batch:  1  loss= tensor(2.3521)  acc= 0.06640625
batch:  2  loss= tensor(2.3536)  acc= 0.09375
batch:  3  loss= tensor(2.3348)  acc= 0.09375
batch:  4  loss= tensor(2.3574)  acc= 0.046875
batch:  5  loss= tensor(2.3695)  acc= 0.0546875
batch:  6  loss= tensor(2.3816)  acc= 0.0703125
batch:  7  loss= tensor(2.3238)  acc= 0.078125
batch:  8  loss= tensor(2.3502)  acc= 0.05078125
batch:  9  loss= tensor(2.3966)  acc= 0.07421875
batch:  10  loss= tensor(2.3653)  acc= 0.08203125
batch:  11  loss= tensor(2.3879)  acc= 0.07421875
batch:  12  loss= tensor(2.3749)  acc= 0.0859375
batch:  13  loss= tensor(2.3650)  acc= 0.0703125
batch:  14  loss= tensor(2.3397)  acc= 0.07421875
batch:  15  loss= tensor(2.4153)  acc= 0.06640625
batch:  16  loss= tensor(2.3088)  acc= 0.1015625
batch:  17  loss= tensor(2.3553)  acc= 0.08203125
batch:  18  loss= tensor(2.3379)  acc= 0.078125
batch:  19  loss= tensor(2.3313)  acc= 0.08203125
batch:  20  loss= tensor(2.

(3) (a) Without any use of the torch.optim package implement from scratch mini-batch Stochastic Gradient Descent training to minimize the loss  $\frac{1}{N}\sum_i^N CrossEntropy(f(x_i),y_i)$ over the MNIST dataset. Use a minibatch size of 128 and a learning rate of 0.01 and run training for 20 epochs.  

You will use torch autograd features (e.g. .backward()) to obtain the gradients given each mini-batch at each parameter and then modify the existing parameters based on this gradient. 

Store the losses and training of each minibatch and plot each of these (with iterations (not epochs) as the x-axis). You can optionally smooth out these plots over 20-100 iteration window of your choosing to make them cleaner to read. Compute and report the final test accuracy as well at the end of the 20 epochs. 

You should end up with 2 plots and a final test accuracy. 


In [ ]:
import torch


train_losses = [] # use to append the avg loss for each minibatch 
train_accs = [] # use to append the avg acc of minibatch

alpha=0.01


for epoch in range(20):
  for ... #Iterate over dataset
    #Compute the gradient for the minibatch

    #Update the model parameters by w_t-alpha*\grad_{w_t}(f(w_t))

    #The following code will clear the gradient buffers for the next iteration
    for (_,param) in param_dict.items():
      if param.grad is not None: 
        param.grad.detach_() 
        param.grad.zero_()
    
    #Update loss and acc tracking 

#Plot the train loss and acc

#Evaluate on the test set

(b) Modify the above code to perform minibatch SGD with momentum. Use a momentum of $\mu=0.9$ and learning rate $\alpha=0.01$. Use the following formulation of momentum: 

$g = \nabla_wCE(f(w_t,X),Y)$  gradient estimate with mini-batch

$v_{t+1} = \mu * v_t + g$ 

$w_{t+1} = w - \alpha*v_{t+1}$ 

Obtain the same plots as before and a final test accuracy

In [ ]:
#Reinitialize your network to random!
initialize_nn(param_dict)

losses = [] # use to append the avg loss for each minibatch 
train_acc = [] # use to append the avg acc of minibatch

alpha=0.01
mu=0.9

for epoch in range(20):
  for ... #Iterate over dataset
    #Compute the gradient for the minibatch

    #Update the model parameters as described above

    #The following code will clear the gradient buffers for the next iteration
    for (_,param) in param_dict.items():
      if param.grad is not None: 
        param.grad.detach_() 
        param.grad.zero_()

    #Update loss and acc tracking 

#Plot the train loss and acc

#Evaluate on the test set

(4) Repeat (3) but now using the package torch.optim.SGD to perform SGD (a) and SGD with momentum (b). Use the same learning rate in the case of (a) and the same learning rate and momentum in the case of (b)


https://pytorch.org/docs/stable/optim.html?highlight=torch%20optim%20sgd#torch.optim.SGD


Plot the same training curves and accuracy curves. Overlay this with their respective implementation from (3). Report the final accuracy.

You should end up with 4 plots (train acc,train loss)x(sgd, sgd+momentum) each containing two curves (your implementation from (3) and the torch.optim result). Due to random initialization and random training order you will not get identical results to (3) but the curves and final accuracies should end up relatively close. 


In [ ]:
#Answer for SGD
import torch
#Make sure to reinitialize your network to random before starting training
initialize_nn(param_dict)

#optim.SGD takes a list of parameters which you can get from your dictionary as follows
parameter_list = param_dict.values()

optimizer = torch.optim.SGD(parameter_list, lr=alpha)




In [ ]:
#Answer for SGD+momentum